# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag ,ne_chunk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier


from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle


In [2]:
# load data from database
engine = create_engine('sqlite:///DR.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df.message
Y = df.iloc[:,4:]

In [3]:
# checking if all shape
df.shape , X.shape , Y.shape

((26028, 39), (26028,), (26028, 35))

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''
    toknizing text messges 
    '''
    # normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ",text.lower())
    # tokenize text
    words = word_tokenize(text)
    # remove stop words
    words = [w for w in words if w not in stopwords.words('English')] 
    # reduce words to their root form
    
    
#     lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]



    # reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]



    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
# train pipeline
pipeline.fit(X_train,y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# test the model
y_pred = pipeline.predict(X_train)
# report the f1 score, precision and recall
print(classification_report(y_train.values, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      0.99      0.99     11972
               request       1.00      0.93      0.96      2698
                 offer       1.00      0.71      0.83        75
           aid_related       1.00      0.96      0.98      6479
          medical_help       1.00      0.83      0.91      1237
      medical_products       1.00      0.84      0.91       784
     search_and_rescue       1.00      0.77      0.87       404
              security       0.99      0.74      0.85       266
              military       1.00      0.87      0.93       509
                 water       1.00      0.92      0.96      1000
                  food       1.00      0.93      0.96      1750
               shelter       1.00      0.91      0.95      1387
              clothing       1.00      0.88      0.94       253
                 money       1.00      0.81      0.90       345
        missing_people       0.99      

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [9]:
# test the model
y_pred_test = pipeline.predict(X_test)
# report the f1 score, precision and recall
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.87      0.89      0.88      7934
               request       0.78      0.47      0.58      1776
                 offer       0.00      0.00      0.00        43
           aid_related       0.78      0.47      0.58      4381
          medical_help       0.52      0.03      0.05       847
      medical_products       0.69      0.04      0.08       529
     search_and_rescue       0.60      0.01      0.02       320
              security       0.50      0.00      0.01       205
              military       0.52      0.05      0.09       351
                 water       0.85      0.22      0.35       672
                  food       0.83      0.45      0.59      1173
               shelter       0.83      0.20      0.32       927
              clothing       0.77      0.15      0.25       152
                 money       0.73      0.03      0.06       259
        missing_people       1.00      

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=1, n_jobs=-1)
clf_cv = cv.fit(X_train, y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 60.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 246.8min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 309.8min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# test the model
y_pred_cv = cv.predict(X_test)
# report the f1 score, precision and recall
print(classification_report(y_test.values, y_pred_cv, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.85      0.93      0.89      7934
               request       0.84      0.49      0.62      1776
                 offer       0.00      0.00      0.00        43
           aid_related       0.81      0.57      0.67      4381
          medical_help       0.62      0.02      0.03       847
      medical_products       0.84      0.03      0.06       529
     search_and_rescue       0.71      0.02      0.03       320
              security       0.50      0.00      0.01       205
              military       0.59      0.07      0.12       351
                 water       0.91      0.25      0.39       672
                  food       0.86      0.44      0.59      1173
               shelter       0.89      0.23      0.37       927
              clothing       0.75      0.12      0.20       152
                 money       1.00      0.01      0.02       259
        missing_people       0.00      

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [16]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 5,
 'clf__n_estimators': 40}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
pipeline_1 = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', DecisionTreeClassifier())
                     ])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y.values, test_size=0.4, random_state=42)
# train pipeline
clf_1 = pipeline_1.fit(X_train,y_train)

In [33]:
# test the model
y_pred_test_1 = pipeline_1.predict(X_test)
# report the f1 score, precision and recall
print(classification_report(y_test, y_pred_test, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.87      0.89      0.88      7934
               request       0.78      0.47      0.58      1776
                 offer       0.00      0.00      0.00        43
           aid_related       0.78      0.47      0.58      4381
          medical_help       0.52      0.03      0.05       847
      medical_products       0.69      0.04      0.08       529
     search_and_rescue       0.60      0.01      0.02       320
              security       0.50      0.00      0.01       205
              military       0.52      0.05      0.09       351
                 water       0.85      0.22      0.35       672
                  food       0.83      0.45      0.59      1173
               shelter       0.83      0.20      0.32       927
              clothing       0.77      0.15      0.25       152
                 money       0.73      0.03      0.06       259
        missing_people       1.00      

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [1]:
m = pickle.dumps('clf_cv')

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.